In [ ]:
! pip install geopandas

In [ ]:
import s3fs
import mlflow
import re
from astrovision.data import SatelliteImage
import geopandas as gpd
from astrovision.plot import plot_images
# mc cp --recursive  s3/projet-slums-detection/ilots/ ilots

In [ ]:
%env MLFLOW_TRACKING_URI=https://projet-slums-detection-128833.user.lab.sspcloud.fr
%env MLFLOW_S3_ENDPOINT_URL=https://minio.lab.sspcloud.fr

# aller dans mlflow cliquer sur un modl et cliquer sur register pour créer ces variables
model_name = "Segmentation"
model_version = "7"
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

### Chargement des îlots

Ya plus les IOU ?
stocvkage moyenne variances dans les artifact MLFlow
Ici je vais charger une table de contour géométriique d'ilot et créer une fonction qui à partir d'un contour polygonal d'îlot  et d'un modèle :

1) récupère les images s'intersectant avec l'îlot (une année donnée) via leur nom (cooronnées du coin incluse dans le nom)
2) un modèle vient faire des prédictions 01 ou en proba sur les images récupérées
3) On somme les probabilités ou les 1 contenus dans l'ilôt (partie la plus subtile ? peut on cropper un raster ? je l'espère)
4) on peut donc attribuer à l'îlot uner surface batie prédite par le modèle
5) se permettre une représentation cartographique en représentant sur un même graphe le fond de carte donné par les patchs qui entour e complètement l'ilot (zone rectangulaire), les prédictions du modèle, et lîlot en question (ou bien d'une prédiction de réseau de neurones en probabilités) somme les1 du masque dans le contour de l'ilot

Je travaille à partir du jeu de test d'images de Mayotte dans un premeir temps ppour avoir des masques prédits hors modèle et des ilots martiniques

Petites remarques comme ça : Quand on est en prod on veut toutes les images du coup dans l'idée plutôt chopper les grosses images qui s'intersectent dans l'ilot, les spliter faire les prédictions et ne garder que les points dans l'ilot : moralité On repart des grosses images m^me si l'intersection est moins fine 
Problème d'EPSG

PB : l'installation de requirements marche pas

In [ ]:
si = SatelliteImage.from_raster(
    file_path="/vsis3/projet-slums-detection/data-raw/PLEIADES/MAYOTTE/2020/ORT_2020052526656219_0505_8595_U38S_8Bits.jp2",
    dep=None,
    date=None,
    n_bands=3,
)

si.plot([0, 1, 2])

 - ok dans  un premier temps je vais prendre l'le nom de l'image  récupérer les coordonnées
 - les transformer en point
 - et trouver quel ilot intersecte l'image


mc cp --recursive  s3/projet-slums-detection/ilots/ ilots

In [ ]:
# Load a shapefile
fp = "ilots/ilots_976.shp"
ilots_mayotte = gpd.read_file(fp)
ilots_mayotte.head()

Je vais sélectionner les images qui tombent dans un ilot (ie où il y a intersection) et voir si je ne peux pas mettre 0 partou à l'extérieur de l'ilot

In [ ]:
!AWS_ACCESS_KEY_ID=`vault kv get -field=ACCESS_KEY_ID onyxia-kv/projet-slums-detection/s3` && export AWS_ACCESS_KEY_ID
!AWS_SECRET_ACCESS_KEY=`vault kv get -field=SECRET_ACCESS_KEY onyxia-kv/projet-slums-detection/s3` && export AWS_SECRET_ACCESS_KEY
!unset AWS_SESSION_TOKEN

In [ ]:
# Create an S3 filesystem object
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://" + "minio.lab.sspcloud.fr"})
# Specify your bucket name and directory path
bucket_name = "projet-slums-detection"
directory_path = "data-raw/PLEIADES/MAYOTTE/2022/"

# Create the full path
full_path = f"{bucket_name}/{directory_path}"

# List files in the specified directory
files = fs.ls(full_path)

print(files[:6])

In [ ]:
import pandas as pd
from pyproj import Transformer
from shapely.geometry import Polygon


# Function to extract coordinates from filename, adjusted for meters
def extract_coordinates(filename):
    matches = re.findall(r"\d{4}", filename)
    if len(matches) >= 2:
        return int(matches[-2]) * 1000, int(matches[-1]) * 1000
    return None, None


# Initialize the transformer from EPSG:4471 to WGS84
transformer = Transformer.from_crs("EPSG:4471", "EPSG:4326")

# Define the length of the square's side (in meters)
side_length = 1000  # Adjust as needed

# Process filenames and transform coordinates to create squares
data = []
for filename in files:
    x, y = extract_coordinates(filename)

    if x and y:
        # Define the square's corners in EPSG:4471
        top_left = (x, y)
        top_right = (x + side_length, y)
        bottom_right = (x + side_length, y - side_length)
        bottom_left = (x, y - side_length)

        # Create a polygon from these corners
        square = Polygon([top_left, top_right, bottom_right, bottom_left])

        # Transform each corner to latitude and longitude
        transformed_corners = [
            (lat, lon)
            for lon, lat in [transformer.transform(*corner) for corner in square.exterior.coords]
        ]

        # Create a new polygon with the transformed coordinates
        square_lat_lon = Polygon(transformed_corners)

        # Store the filename and the polygon
        data.append({"filename": filename, "geometry": square_lat_lon})

# Create a DataFrame
df = pd.DataFrame(data)

filename_to_polygon = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
filename_to_polygon.crs = "EPSG:4326"
# This DataFrame now contains the filenames and the corresponding square polygons in latitude and longitude
filename_to_polygon.head()

In [ ]:
intersection_ilot_filename = gpd.sjoin(
    ilots_mayotte, filename_to_polygon, how="inner", predicate="intersects"
)
print(intersection_ilot_filename)

TO DO : représenter les ilots et les images sur un meme graph

In [ ]:
filename_ilot = intersection_ilot_filename.loc[
    intersection_ilot_filename["code_ilot"] == "0201"
].filename
filename_ilot

Objectif : superposer la mosaique d'imlage extraite et le cntour d'ilot
Superposer les prédictions du modèle
comment cropper un raster avec un polygone ?

#filename_ilot , créer la liste de si et faire la mosaique avec astrovision

In [ ]:
list_si = [
    SatelliteImage.from_raster(
        file_path="/vsis3/" + fp,
        dep=None,
        date=None,
        n_bands=3,
    )
    for fp in filename_ilot
]

In [ ]:
list_si[2].plot([0, 1, 2])

In [ ]:
plot_images(list_si[:2], [0, 1, 2])